# Zadanie 2 – Klasyfikacja intencji użytkownika
W tym ćwiczeniu stworzymy prosty klasyfikator intencji wypowiedzi użytkownika przy użyciu tokenizacji, sekwencji i sieci neuronowej.

In [ ]:
# Instalacja wymaganych bibliotek (jeśli potrzebne)
# !pip install tensorflow scikit-learn pandas
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd

## Dane wejściowe

In [ ]:
data = {
    "text": [
        "Chciałbym zamówić pizzę",
        "Pokaż mi pogodę w Warszawie",
        "Jakie są twoje godziny otwarcia?",
        "Czy mogę zmienić termin wizyty?",
        "Potrzebuję pomocy z logowaniem",
        "Zarezerwuj stolik na jutro",
        "Wyślij mi fakturę",
        "Jak mogę się z tobą skontaktować?",
        "Jaka jest cena tego produktu?"
    ],
    "intent": [
        "zamówienie_jedzenia",
        "sprawdzenie_pogody",
        "informacja_godziny",
        "zmiana_terminu",
        "wsparcie_logowanie",
        "rezerwacja",
        "żądanie_faktury",
        "kontakt",
        "informacja_cena"
    ]
}
df = pd.DataFrame(data)
df

## Przygotowanie danych

In [ ]:
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
padded = pad_sequences(sequences, maxlen=10, padding='post')

label2id = {label: i for i, label in enumerate(df['intent'].unique())}
id2label = {v: k for k, v in label2id.items()}
labels = df['intent'].map(label2id).values

## Budowa modelu

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=1000, output_dim=16, input_length=10),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(label2id), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## Trening modelu

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

## Predykcja nowych intencji

In [ ]:
test_sentences = [
    "Czy mogę dostać fakturę?",
    "Potrzebuję pomocy przy logowaniu",
    "Pokaż pogodę na jutro",
    "Zamów dla mnie pizzę na jutro"
]
test_seq = tokenizer.texts_to_sequences(test_sentences)
test_pad = pad_sequences(test_seq, maxlen=10, padding='post')
pred = model.predict(test_pad)
for i, p in enumerate(pred):
    intent = id2label[p.argmax()]
    print(f"{test_sentences[i]} -> {intent}")